In [1]:
from vtk import *

In [2]:
# Load data
#######################################
reader = vtkXMLImageDataReader()
reader.SetFileName('Data/Isabel_2D.vti')
reader.Update()
data = reader.GetOutput()

In [34]:
data.GetCell(0).GetPointIds().GetId(0)

0

In [20]:
import vtk

def extract_isocontour(input_filename, output_filename, isovalue):
    # Load the image data
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(input_filename)
    reader.Update()
    image_data = reader.GetOutput()
    
    # Get the number of cells and points
    num_cells = image_data.GetNumberOfCells()
    num_points = image_data.GetNumberOfPoints()
    print("Num of cells",num_cells)
    print("Num of points",num_points)
    print("Num of points in cell 1",image_data.GetCell(1).GetNumberOfPoints())
    print("Point Ids in cell 1",image_data.GetCell(1).GetPointIds().GetId(0),image_data.GetCell(1).GetPointIds().GetId(1),image_data.GetCell(1).GetPointIds().GetId(2),image_data.GetCell(1).GetPointIds().GetId(3))
    print("Val at cell 1",image_data.GetPointData().GetScalars().GetTuple1(1))
    # Get the scalar values of each point
    values = []
    for i in range(num_points):
        values.append(image_data.GetPointData().GetScalars().GetTuple1(i))
    print("len of values",len(values))
    print("Range of values",min(values),max(values))
    # Create arrays for storing the contour segments
    contour_points = vtk.vtkPoints()
    contour_lines = vtk.vtkCellArray()
    contour_segments = []
    
    # Loop through each cell and determine the contour segments
    for i in range(num_cells):
        cell = image_data.GetCell(i)
        num_cell_points = cell.GetNumberOfPoints()
        cell_points = cell.GetPointIds()
        
        # Find the point pairs that lie on either side of the isovalue
        # for j in range(num_cell_points - 1):
        c = 0
        l = []
        for j in [0,1,3,2]:
            if c==2:
                break
            # print(j)
            k = 1
            if j==1:
                k=3
            elif j==3:
                k = 2 
            elif j==2:
                k = 0
            p1 = cell_points.GetId(j)
            p2 = cell_points.GetId(k)
            
            if (values[p1] <= isovalue and values[p2] > isovalue) or (values[p1] > isovalue and values[p2] <= isovalue):
                # Interpolate the points along the line segment
                c += 1
                fraction = (isovalue - values[p1]) / (values[p2] - values[p1])
                x1, y1, z1 = image_data.GetPoint(p1)
                x2, y2, z2 = image_data.GetPoint(p2)
                x = x1 + fraction * (x2 - x1)
                y = y1 + fraction * (y2 - y1)
                z = 25
                # Add the interpolated point to the contour points
                contour_points.InsertNextPoint(x, y, z)
                l.append(contour_points.GetNumberOfPoints() - 1)
                # contour_segments.append((len(contour_segments), len(contour_segments) + 1))
        if l:
            line = vtk.vtkLine()
            line.GetPointIds().SetId(0, l[0])
            line.GetPointIds().SetId(1, l[1])
            contour_lines.InsertNextCell(line)
    
    # Create the polydata object
    polydata = vtk.vtkPolyData()
    polydata.SetPoints(contour_points)
    
    # Convert the contour segments list to a vtkCellArray
    # contour_lines = vtk.vtkCellArray()
    # print(contour_segments)
    # for segment in contour_segments: #[(0,1),(1,2),(2,3)]
    #     line = vtk.vtkLine()
    #     line.GetPointIds().SetId(0, segment[0])
    #     line.GetPointIds().SetId(1, segment[1])
    #     contour_lines.InsertNextCell(line)
        
    polydata.SetLines(contour_lines)
    
    mapper = vtkPolyDataMapper()
    mapper.SetInputData(polydata)
    actor = vtkActor()
    actor.SetMapper(mapper)

    ### Setup render window, renderer, and interactor
    ##################################################
    renderer = vtkRenderer()
    renderer.SetBackground(0.5,0.5,0.5)
    render_window = vtkRenderWindow()
    render_window.SetSize(1400,1000)
    render_window.AddRenderer(renderer)
    render_windowInteractor = vtkRenderWindowInteractor()
    render_windowInteractor.SetRenderWindow(render_window)
    render_windowInteractor.SetInteractorStyle(vtkInteractorStyleTrackballCamera())
    renderer.AddActor(actor)

    render_window.Render()
    render_windowInteractor.Start()
    print(polydata)
    # Write the polydata to a file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_filename)
    writer.SetInputData(polydata)
    writer.Write()
    
    print("Isocontour extraction complete. Output written to", output_filename)


NameError: name 'polydata' is not defined

In [21]:
extract_isocontour('Data/Isabel_2D.vti', "k_c.vtp", 500)

Num of cells 62001
Num of points 62500
Num of points in cell 1 4
Point Ids in cell 1 1 2 251 252
Val at cell 1 474.79827880859375
len of values 62500
Range of values -1434.8590087890625 630.5694580078125
vtkPolyData (0x3867250)
  Debug: Off
  Modified Time: 9387410
  Reference Count: 3
  Registered Events: (none)
  Information: 0x371c9e0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 9396549
  Field Data:
    Debug: Off
    Modified Time: 9387400
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 1800
  Number Of Cells: 900
  Cell Data:
    Debug: Off
    Modified Time: 9387403
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x33c7a50)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x374c4b0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tupl

In [1]:
import vtk

def extract_isocontour_filter(input_filename, output_filename, isovalue):
    # Read the input VTKImageData file
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(input_filename)
    reader.Update()
    image_data = reader.GetOutput()
    
    # Extract geometry from the image data
    geometryFilter = vtk.vtkImageDataGeometryFilter()
    geometryFilter.SetInputData(image_data)
    geometryFilter.Update()
    poly_data = geometryFilter.GetOutput()
    
    # Extract the isocontour
    contourFilter = vtk.vtkContourFilter()
    contourFilter.SetInputData(poly_data)
    contourFilter.SetValue(0, isovalue)
    contourFilter.Update()
    poly_data = contourFilter.GetOutput()
    
    # Write the output VTKPolyData file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_filename)
    writer.SetInputData(contourFilter.GetOutput())
    writer.Write()

if __name__ == "__main__":
    extract_isocontour_filter('Data/Isabel_2D.vti', "k_f.vtp", 20)


Q2

In [68]:
import vtk
# import sys

# Load the 3D data
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("Data_New/Isabel_3D.vti")
reader.Update()

# Create color transfer function
color_tf = vtk.vtkColorTransferFunction()
color_tf.AddRGBPoint(-4931.54, 0, 1, 1)
color_tf.AddRGBPoint(-2508.95, 0, 0, 1)
color_tf.AddRGBPoint(-1873.9, 0, 0, 0.5)
color_tf.AddRGBPoint(-1027.16, 1, 0, 0)
color_tf.AddRGBPoint(-298.031, 1, 0.4, 0)
color_tf.AddRGBPoint(2594.97, 1, 1, 0)

# Create opacity transfer function
opacity_tf = vtk.vtkPiecewiseFunction()
opacity_tf.AddPoint(-4931.54, 1.0)
opacity_tf.AddPoint(101.815, 0.002)
opacity_tf.AddPoint(2594.97, 0.0)

# Perform volume rendering
mapper = vtk.vtkSmartVolumeMapper()
mapper.SetInputConnection(reader.GetOutputPort())

# Use Phong shading or not
# use_phong = input("Do you want to use Phong shading? (yes/no) ")
use_phong = "yes"

volume = vtk.vtkVolume()
volume_property = vtk.vtkVolumeProperty()
volume_property.SetColor(color_tf)
volume_property.SetScalarOpacity(opacity_tf)
volume_property.ShadeOn()
if use_phong == "yes":
    volume_property.SetAmbient(0.5)
    volume_property.SetDiffuse(0.5)
    volume_property.SetSpecular(0.5)
else:
    volume_property.ShadeOff()
volume.SetMapper(mapper)
volume.SetProperty(volume_property)

# Add an outline to the volume rendered data
outline = vtk.vtkOutlineFilter()
outline.SetInputConnection(reader.GetOutputPort())
outline_mapper = vtk.vtkPolyDataMapper()
outline_mapper.SetInputConnection(outline.GetOutputPort())
outline_actor = vtk.vtkActor()
outline_actor.SetMapper(outline_mapper)

# Create a render window
renderer = vtk.vtkRenderer()
# renderer.SetBackground(1,1,1)
renderer.AddVolume(volume)
renderer.AddActor(outline_actor)

render_window = vtk.vtkRenderWindow()
render_window.SetSize(1000, 1000)
render_window.AddRenderer(renderer)
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

render_window.Render()
render_window_interactor.Start()
